In [1]:
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [2]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue.

with engine.connect() as con:
    rs = con.execute('  SELECT name, revenue \
                        FROM (SELECT facid, f.name, SUM( \
                                CASE WHEN b.memid = 0 THEN f.guestcost * b.slots\
                                ELSE f.membercost * b.slots END) AS revenue \
                            FROM Facilities AS f \
                            INNER JOIN Bookings AS b \
                            USING (facid) \
                            INNER JOIN Members AS m \
                            USING (memid) \
                            GROUP BY f.facid) \
                        WHERE revenue < 1000 \
                        ORDER BY revenue')
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

df

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [3]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

with engine.connect() as con:
    rs = con.execute("  SELECT (mem.surname || ', ' || mem.firstname) AS member, \
                                CASE WHEN mem.recommendedby > 0 THEN (rec.surname || ', ' || rec.firstname) \
                                ELSE 'Nobody' END AS recommended_by\
                        FROM Members as mem \
                        LEFT JOIN Members as rec\
                        ON mem.recommendedby = rec.memid\
                        WHERE mem.memid <> 0\
                        ORDER BY member")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

df

,member,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",Nobody
9,"Farrell, Jemima",Nobody


In [4]:
# Q12: Find the facilities with their usage by member, but not guests

with engine.connect() as con:
    rs = con.execute("  SELECT member_name, facility_name, \
	                        COUNT(*) AS number_of_bookings \
                        FROM (SELECT memid, facid, f.name AS facility_name, \
                              m.surname || ', ' || m.firstname AS member_name \
                              FROM Members AS m \
                              INNER JOIN Bookings as b \
                              USING (memid) \
                              INNER JOIN Facilities AS f \
                              USING (facid) \
                              WHERE b.memid <> 0) AS sub \
                        GROUP BY facid, memid \
                        ORDER BY member_name, facility_name;")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

df

,member_name,facility_name,number_of_bookings
0,"Bader, Florence",Badminton Court,9
1,"Bader, Florence",Massage Room 2,2
2,"Bader, Florence",Pool Table,23
3,"Bader, Florence",Snooker Table,33
4,"Bader, Florence",Squash Court,2
...,...,...,...
197,"Tupperware, Hyacinth",Squash Court,1
198,"Worthington-Smyth, Henry",Badminton Court,4
199,"Worthington-Smyth, Henry",Massage Room 1,1
200,"Worthington-Smyth, Henry",Pool Table,33


In [5]:
# Q13: Find the facilities usage by month, but not guests

with engine.connect() as con:
    rs = con.execute("  SELECT month, facility_name, \
	                        COUNT(*) AS number_of_bookings \
                        FROM (SELECT facid, f.name AS facility_name, \
                              strftime('%m', b.starttime) AS month \
                              FROM Members AS m \
                              INNER JOIN Bookings as b \
                              USING (memid) \
                              INNER JOIN Facilities AS f \
                              USING (facid) \
                              WHERE b.memid <> 0) AS sub \
                        GROUP BY facid, month \
                        ORDER BY month, facility_name;")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

df

,month,facility_name,number_of_bookings
0,07,Badminton Court,51
1,07,Massage Room 1,77
2,07,Massage Room 2,4
3,07,Pool Table,103
4,07,Snooker Table,68
5,07,Squash Court,23
6,07,Table Tennis,48
7,07,Tennis Court 1,65
8,07,Tennis Court 2,41
9,08,Badminton Court,132
